# Prática guiada - Join com Pandas.

## PARTE I.

#### Algumas das operações mais interessantes com dados vêm da combinação de diferentes fontes de dados. Elas podem ser:

    1. simples concatenações de dados de conjuntos de dados diferentes
    2. operações mais parecidas com um join ou um merge em um banco de dados

#### Tanto `Series` quanto `DataFrames` foram construídos levando em conta essas operações e incluem funções e métodos para realizá-las de forma rápida e simples.

#### Vamos ver duas operações: [`pd.append()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html) e [`pd.concat()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html).

In [117]:
import pandas as pd
import numpy as np

#### Vamos criar uma função que cria um `DataFrame` para simplificarmos alguns passos:

In [118]:
def make_df(cols, ind):
    """
    Quickly make a DataFrame
    """
    data = {c: [str(c) + str(i) for i in ind] for c in cols}    
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


#### Também vamos [criar uma classe](https://realpython.com/python3-object-oriented-programming/) para podermos imprimir “side by side” diferentes `DataFrame`. A ideia é conseguir visualizar algumas das operações que vamos realizar.

#### a função [`str.join()`](https://docs.python.org/3/library/stdtypes.html#str.join) retorna uma concatenação de elementos, ligados pelo valoe em `str`.


[`self.template.format()`]()
[`eval()`]()


[`*args`](https://medium.com/rafaeltardivo/python-entendendo-o-uso-de-args-e-kwargs-em-fun%C3%A7%C3%B5es-e-m%C3%A9todos-c8c2810e9dc8)

[`self`](https://www.geeksforgeeks.org/self-in-python-class/)

In [119]:
'\n'.join(('Esse', 'é', 'o', 'resultado', 'da', 'função', 'join'))

'Esse\n\xc3\xa9\no\nresultado\nda\nfun\xc3\xa7\xc3\xa3o\njoin'

In [120]:
class display(object):
    """ 
    Display HTML representation of multiple objects
    """
    template = """<div style="float: left; padding: 10px;"> 
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1} 
    </div>"""

    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_()) for a in self.args)

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a)) for a in self.args)
    

## Concatenação simples com ``pd.concat``

#### A função [`pd.concat()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html), permite fazer concatenações simples de diferentes `Series`.

#### Ela tem uma sintaxe semelhante à sua análoga em Numpy [`np.concatenate`](https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html), mas também contém algumas opções adicionais:

```python
pd.concat(objs, axis = 0, join = 'outer', join_axes = None, ignore_index = False,
keys = None, levels = None, names = None, verify_integrity = False, copy = True)
```

In [121]:
ser1 = pd.Series(['A', 'B', 'C'], index = [1, 2, 3])
#ser1
ser2 = pd.Series(['D', 'E', 'F'], index = [4, 5, 6])
#ser2

pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

#### Ela também permite concatenar objetos de maior dimensionalidade, como `DataFrame`:

In [122]:
df1 = make_df('AB', [1, 2])
#df1
df2 = make_df('AB', [3, 4])
#df2

display('df1', 'df2', "pd.concat([df1, df1], axis='rows')")

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
1,A1,B1
2,A2,B2


#### Por omissão, a concatenação é feita no sentido das linhas do ``DataFrame`` (i.e., ``axis=0``), mas é possível especificar o eixo sobre o qual fazer a concatenação:

In [123]:
df3 = make_df('AB', [0, 1])
#df3
df4 = make_df('CD', [0, 1])
df4
display('df3', 'df4', "pd.concat([df3, df4], axis='columns')")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis='columns')
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

### Índice duplicados

#### Lembrem que numpy também apresenta um método de concatenação.

#### Uma diferença importante entre `np.concatenate` e `pd.concat` é que a concatenação do Pandas preserva os índices, ainda se o resultado envolver índices duplicados:

In [124]:
x = make_df('AB', [0, 1])
#x
y = make_df('AB', [2, 3])
y
#y.index = x.index # índices duplicados!
print(x.index)
print(y.index)
display('x','y')

Int64Index([0, 1], dtype='int64')
Int64Index([2, 3], dtype='int64')


,A,B
0,A0,B0
1,A1,B1
,A,B
2,A2,B2
3,A3,B3


In [125]:
#display('x', 'y', 'np.concatenate([x, y])')
display('x', 'y', 'pd.concat([x, y])')
#np.concatenate([x, y])
#pd.concat([x, y])

,A,B
0,A0,B0
1,A1,B1
,A,B
2,A2,B2
3,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### Embora `DataFrame` permita a existência de índices duplicados é preferível evitar.

#### Verificando a existência de índices duplicados

#### Podemos conferir se há índices solapados no resultado de `pd.concat()` usando uma ``verify_integrity`` flag.

* Colocando True, a concatenação marcará uma exceção se houver algum índice duplicado:

#### O parâmetro `verify_integrity` da função `pd.concat([x, y], verify_integrity = True)` verifica se o novo eixo concatenado contém duplicatas. Por via de regra, para não receber esse comportamento diante de um erro, podemos gerar uma [exceção](https://www.programiz.com/python-programming/exception-handling) para mostrá-lo ao usuário.

In [126]:
try:
    pd.concat([x, y], verify_integrity = True)
except ValueError as e:
    print("ValueError:", e)

### Ignorando o índice:

#### Em alguns casos o índice não tem importância ou é possível ignorá-lo, para isso usamos parâmetro `ignore_index`. Com `ignore_index = True`, a função não usa os valores de índice ao longo do eixo de concatenação.

In [127]:
display('x', 'y', 'pd.concat([x, y], ignore_index = True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
2,A2,B2
3,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


### O método `append`

#### Sendo que a concatenação de arrays é bem comum, `Series` e `DataFrame` têm um método [`.append()`](https://docs.python.org/3/tutorial/datastructures.html). Por exemplo, em vez de chamar `pd.concat( [df1, df2] )`, é possível chamar `df1.append(df2)`, que é mais simples:

In [128]:
display('df1', 'df2', 'df1.append(df2)')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [129]:
df1.append(df2)

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [130]:
df1

,A,B
1,A1,B1
2,A2,B2


#### **IMPORTANTE:** considere que, diferentemente do método `append` de listas,o `append()` do Pandas não altera o objeto original. Ele gera um novo objeto com os dados combinados.

* Como isso envolve a criação de um novo índice e um novo conjunto de dados, `append` pode não ser o melhor método se o plano for concatenar muitos datasets consecutivos.

* Nesses casos é melhor usar a função `pd.concat()`.

# PARTE II 

## Tipos de relacionamentos

#### Uma das características mais valiosas da biblioteca Pandas é sua funcionalidade para realizar joins em memória de forma eficiente.

#### O método merge() permite trabalhar com objetos que apresentam diferentes tipos de relacionamentos:

    1. Um a um.
    2. Muitos a um.
    3. Muitos a muitos.
 

### 1. Join um a um.

In [131]:
import pandas as pd
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'], 
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']
                   }
                  )
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'], 
                    'hire_date': [2004, 2008, 2012, 2014]
                   }
                  )
print(df1)
print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


#### Vemos que cada funcionário tem um único grupo e uma única data de contratação. Combinamos os dataframes usando [`pd.merge()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html). 

#### Observe que a função merge encontrou a única coluna compartilhada por ambos os dataframes `"employee"`. A função exige que a coluna tenha o mesmo nome nos dois df.

In [132]:
df3 = pd.merge(df1, df2)
print(df3)

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


### 2. Join um a muitos.

In [133]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'], 
                    'supervisor': ['Carly', 'Guido', 'Steve']
                   }
                  )
print(df4)

         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve


#### Observe que cada supervisor pertence a UM grupo que pode ter MUITOS funcionários.

#### No join entre funcionários e supervisores, os funcionários aparecerão uma única vez, mas os supervisores podem se repetir.

In [134]:
pd.merge(df3, df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


### 3. Joint muitos a muitos.

In [135]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting', 'Engineering', 'Engineering', 'HR', 'HR'], 
                    'skills': ['math', 'spreadsheets', 'coding', 'linux','spreadsheets', 'organization']
                   }
                  )
print(df5)
print(df1)

         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR


#### Observe que cada grupo tem MUITOS skills associados e que também podem pertencer a ele MUITOS funcionários. Portanto, o join entre a tabela de skills e a de funcionários é de MUITOS a MUITOS. Vamos ver no resultado que tanto os skills quanto os funcionários podem se repetir.

In [136]:
df6 = pd.merge(df1, df5)
print(df6)

  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


## Joins por diferentes colunas.

#### Pode acontecer que nos nossos dataframes não tenham uma única coluna com o mesmo nome em ambas as tabelas para poder realizar o join. Para resolver esse problema, o Pandas implementa os parâmetros `on`, `right_on` e `left_on`, onde podemos especificar com quais colunas vamos unir os dados.

### 1. Join com `on`.

In [137]:
print(df1)
print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [138]:
pd.merge(df1, df2, on = 'employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### 2. Join com `left_on` e `right_on`.

In [139]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 
                    'salary': [70000, 80000, 120000, 90000]
                   }
                  )
print(df1)
print(df3)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000


In [140]:
pd.merge(df1, df3, left_on = "employee", right_on = "name")

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


### 3. Join com mais de uma coluna.

In [141]:
df7 = pd.DataFrame({'group': ['Accounting', 'Accounting', 'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux','spreadsheets', 'organization'],
                    'tools': ['calculator','desktop computer','laptop computer','server','desktop computer','board']
                   }
                  )
print(df6)
print(df7)

  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization
         group        skills             tools
0   Accounting          math        calculator
1   Accounting  spreadsheets  desktop computer
2  Engineering        coding   laptop computer
3  Engineering         linux            server
4           HR  spreadsheets  desktop computer
5           HR  organization             board


#### Agora podemos ver as ferramentas por cada funcionário.

In [142]:
pd.merge(df6, 
         df7, 
         left_on = ['group','skills'], 
         right_on = ['group','skills']
        )

,employee,group,skills,tools
0,Bob,Accounting,math,calculator
1,Bob,Accounting,spreadsheets,desktop computer
2,Jake,Engineering,coding,laptop computer
3,Lisa,Engineering,coding,laptop computer
4,Jake,Engineering,linux,server
5,Lisa,Engineering,linux,server
6,Sue,HR,spreadsheets,desktop computer
7,Sue,HR,organization,board


#### Como os nomes das colunas são iguais, usar apenas `on` é equivalente.

In [143]:
pd.merge(df6,df7,on=['group','skills'])

,employee,group,skills,tools
0,Bob,Accounting,math,calculator
1,Bob,Accounting,spreadsheets,desktop computer
2,Jake,Engineering,coding,laptop computer
3,Lisa,Engineering,coding,laptop computer
4,Jake,Engineering,linux,server
5,Lisa,Engineering,linux,server
6,Sue,HR,spreadsheets,desktop computer
7,Sue,HR,organization,board


## Tipos de joins.

In [144]:
import pandas as pd
from IPython.display import display
from IPython.display import Image

### 1. Left joins

In [145]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'],
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']
}

df_a = pd.DataFrame(raw_data, 
                    columns = ['subject_id', 'first_name', 'last_name']
                   )
df_a

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


In [146]:
raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'],
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']
}
df_b = pd.DataFrame(raw_data, 
                    columns = ['subject_id', 'first_name', 'last_name']
                   )
df_b

,subject_id,first_name,last_name
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


#### Usar o valor `left` na forma do `join` produz uma lista completa das linhas de `df_a` com as linhas coincidentes de `df_b`. Se não houver coincidência, as colunas que vêm de `df_b` serão nulas.

In [147]:
pd.merge(df_a, 
         df_b, 
         on = 'subject_id', 
         how = 'left'
        )

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,1,Alex,Anderson,NaN,NaN
1,2,Amy,Ackerman,NaN,NaN
2,3,Allen,Ali,NaN,NaN
3,4,Alice,Aoni,Billy,Bonder
4,5,Ayoung,Atiches,Brian,Black


### Check: Qual seria o resultado de trocar left por right?

In [148]:
pd.merge(df_a, 
         df_b, 
         on = 'subject_id', 
         how = 'right'
        )

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,4,Alice,Aoni,Billy,Bonder
1,5,Ayoung,Atiches,Brian,Black
2,6,NaN,NaN,Bran,Balwner
3,7,NaN,NaN,Bryce,Brice
4,8,NaN,NaN,Betty,Btisan


### 2. `Inner` e `outer join`.

#### Conforme mencionado acima, usar a forma `outer` (`OUTER JOIN`)  produz um conjunto de todas as linhas em `df_a` e `df_b`.  Todas as colunas terão valores se a linha tiver uma correspondente no outro. Se não houver coincidência, as colunas onde não tinha valor são preenchidas com `null`.

In [149]:
pd.merge(df_a, 
         df_b, 
         on = 'subject_id', 
         how = 'outer'
        )

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,1,Alex,Anderson,NaN,NaN
1,2,Amy,Ackerman,NaN,NaN
2,3,Allen,Ali,NaN,NaN
3,4,Alice,Aoni,Billy,Bonder
4,5,Ayoung,Atiches,Brian,Black
5,6,NaN,NaN,Bran,Balwner
6,7,NaN,NaN,Bryce,Brice
7,8,NaN,NaN,Betty,Btisan


### Check: O que aconteceria se usássemos um inner join?

In [150]:
pd.merge(df_a, 
         df_b, 
         on = 'subject_id', 
         how = 'inner'
        )

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,4,Alice,Aoni,Billy,Bonder
1,5,Ayoung,Atiches,Brian,Black


### Encontrando os casos que aparecem em um dataframe, mas não no outro, com `left join`.

#### Um problema comum que podemos querer resolver é como encontrar os casos presentes em uma tabela, mas não em outra. É possível fazer isso sem dificuldade com um `left join` pelo(s) campo(s) que compõem a chave. 

In [151]:
df1 = pd.DataFrame(data = {'col1' : [1, 2, 3, 4, 5], 
                           'col2' : [10, 11, 12, 13, 14]
                          }
                  ) 
df2 = pd.DataFrame(data = {'col1' : [1, 2, 3], 
                           'col2' : [10, 11, 12]
                          }
                  )
print(df1)
print(df2)

   col1  col2
0     1    10
1     2    11
2     3    12
3     4    13
4     5    14
   col1  col2
0     1    10
1     2    11
2     3    12


#### Acrescentem a ambos os dataframes umas chaves que tomam sempre o mesmo valor para poder comparar depois.

In [152]:
df1['key1'] = 1
df2['key2'] = 1
print(df1)
print(df2)

   col1  col2  key1
0     1    10     1
1     2    11     1
2     3    12     1
3     4    13     1
4     5    14     1
   col1  col2  key2
0     1    10     1
1     2    11     1
2     3    12     1


#### Quando fazemos o `left join`, os valores de `key2` são preenchidos com `null` para aqueles valores de `df2` que não existem em `df1`.

In [153]:
df1 = pd.merge(df1, 
               df2, 
               on = ['col1', 'col2'], 
               how = 'left'
              )
df1

,col1,col2,key1,key2
0,1,10,1,1.0
1,2,11,1,1.0
2,3,12,1,1.0
3,4,13,1,NaN
4,5,14,1,NaN


#### Podemos fazer um subset do resultado do merge para não ficar com aqueles que aparecem em `df1`, mas não em `df2`.

In [154]:
df3 = df1[~(df1.key2 == df1.key1)]
df3 = df3.drop(['key1','key2'], 
               axis = 1
              )
df3

,col1,col2
3,4,13
4,5,14


### Revisão de práticas recomendadas de performance

#### O método `join` tem a mesma sintaxe e as mesmas possibilidades que o método `merge`, mas com a diferença de que ele sempre faz o relacionamento pelo `index`. 

#### Criamos dois Dataframes de tamanho `1,000,000`.

In [155]:
df1 = pd.DataFrame(np.arange(1000000), 
                   columns = ['A']
                  )
df1['B'] = np.random.randint(0, 
                             1000, 
                             (1000000)
                            )
df1.head()

,A,B
0,0,632
1,1,988
2,2,504
3,3,706
4,4,269


In [156]:
df2 = pd.DataFrame(np.arange(1000000), 
                   columns=['A2']
                  )
df2['B2'] = np.random.randint(0, 
                              1000, 
                              (1000000)
                             )
df2.head()

,A2,B2
0,0,263
1,1,134
2,2,569
3,3,844
4,4,651


#### Medimos o tempo de execução do `merge`.

In [157]:
def a():
    x = df1.merge(df2, 
                  how = 'left', 
                  left_on = 'A', 
                  right_on = 'A2'
                 )

In [158]:
%timeit a()

1 loop, best of 3: 1.29 s per loop


#### Medimos o tempo de execução do `join`.

In [159]:
def b():
    x = df1.set_index('A').join(df2.set_index('A2'), 
                                how = 'left'
                               )

In [160]:
%timeit b()

1 loop, best of 3: 297 ms per loop


#### Criamos dois Dataframes de tamanho `1,000,000`.

In [161]:
df1 = pd.DataFrame(np.arange(1000000), 
                   columns = ['A']
                  )
df1['B'] = np.random.randint(0, 
                             1000, 
                             (1000000)
                            )
df2 = pd.DataFrame(np.arange(1000000), 
                   columns = ['A2']
                  )
df2['B2'] = np.random.randint(0, 1000, (1000000)
                             )

In [162]:
def set_indA(df):
     df.set_index('A')

In [163]:
def set_indA2(df):
    df.set_index('A2')

In [164]:
def c():
    df1.join(df2)

In [165]:
%timeit set_indA(df1)

10 loops, best of 3: 30.5 ms per loop


In [166]:
%timeit set_indA2(df2)

10 loops, best of 3: 30.9 ms per loop


In [167]:
%timeit c()

10 loops, best of 3: 25.9 ms per loop


## Trabalho com séries temporais: calcular variações com shift()

#### No trabalho com séries temporais, é muito comum acrescentar ao dataframe uma variável que indica o grau de alteração de certo valor desde a última medição.

#### Criamos o dataframe.

In [168]:
df = pd.DataFrame()

#### Acrescentamos as séries de dados.

In [169]:
df['hora'] = ['10:30','10:31','10:38','10:40','10:41']
df['quantidade'] = [20,20,9,12,12]
df

,hora,quantidade
0,10:30,20
1,10:31,20
2,10:38,9
3,10:40,12
4,10:41,12


In [170]:
df['quantidadeAnterior'] = df['quantidade']
df

,hora,quantidade,quantidadeAnterior
0,10:30,20,20
1,10:31,20,20
2,10:38,9,9
3,10:40,12,12
4,10:41,12,12


#### O método [`.shift()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html) deslocar o índice pelo número desejado de períodos.

In [171]:
df['quantidadeAnterior'] = df['quantidade'].shift(periods = 1)
df

,hora,quantidade,quantidadeAnterior
0,10:30,20,NaN
1,10:31,20,20.0
2,10:38,9,20.0
3,10:40,12,9.0
4,10:41,12,12.0


In [172]:
df['variacao'] = df['quantidade'] - df['quantidadeAnterior']

In [173]:
df

,hora,quantidade,quantidadeAnterior,variacao
0,10:30,20,NaN,NaN
1,10:31,20,20.0,0.0
2,10:38,9,20.0,-11.0
3,10:40,12,9.0,3.0
4,10:41,12,12.0,0.0


### Variações com agrupamento.

#### É muito habitual ter que calcular isso, mas para cada indivíduo ou categoria que seja de interesse estudar.

In [174]:
df = pd.DataFrame()

df['operador'] = ['Q8','Q8','Q8','Q7','Q9','Q9']
df['hora'] = ['10:30','10:31','10:32','10:38','10:40','10:41']
df['quantidade'] = [15,20,10,9,12,12]
df

,operador,hora,quantidade
0,Q8,10:30,15
1,Q8,10:31,20
2,Q8,10:32,10
3,Q7,10:38,9
4,Q9,10:40,12
5,Q9,10:41,12


In [175]:
df['quantidadeAntOperador'] = df.groupby(['operador']
                                          )['quantidade'].transform(lambda x: x.shift()
                                                                   )
df

,operador,hora,quantidade,quantidadeAntOperador
0,Q8,10:30,15,NaN
1,Q8,10:31,20,15.0
2,Q8,10:32,10,20.0
3,Q7,10:38,9,NaN
4,Q9,10:40,12,NaN
5,Q9,10:41,12,12.0


In [176]:
df['variacao_operador'] = df['quantidade'] - df['quantidadeAntOperador']
df

,operador,hora,quantidade,quantidadeAntOperador,variacao_operador
0,Q8,10:30,15,NaN,NaN
1,Q8,10:31,20,15.0,5.0
2,Q8,10:32,10,20.0,-10.0
3,Q7,10:38,9,NaN,NaN
4,Q9,10:40,12,NaN,NaN
5,Q9,10:41,12,12.0,0.0


## Groupby

#### Encontrando o total da 'quantidade' agrupando a coluna 'operador'.

In [177]:
dfGroups = df.groupby("operador")["quantidade"].sum().reset_index()
dfGroups
CountryGroups = sales.groupby("Country")["Revenue"].sum().reset_index()

NameError: name 'sales' is not defined

In [ ]:
dfGroups.sort_values(by = "quantidade", 
                     ascending = False, 
                     inplace = True
                    )
dfGroups

In [ ]:
dfGroups.shape